In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
import pandas_profiling
from matplotlib import pyplot as plt
import sklearn.gaussian_process as gp
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

# Distributions
import scipy

# PyMC3 for Bayesian Inference
import pymc3 as pm

In [ ]:
df = pd.read_csv("/.../Arctic_domain_mean_monthly_1979_2021.csv")
# remove date from the set
df = df.drop('Date', 1)
df.insert(0, 'label', df['sea_ice_extent'])
df


,label,wind_10m,specific_humidity,LW_down,SW_down,rainfall,snowfall,sst,t2m,surface_pressure,sea_ice_extent
0,15604191,5.531398,0.811961,186.687054,3.127880,1.009872,0.892319,273.355237,250.388101,984.633032,15604191
1,16378929,5.328020,0.688896,174.794571,18.541594,0.920831,0.781347,273.121885,247.071202,983.980418,16378929
2,16521089,5.432511,0.916124,190.741933,67.690429,0.983327,0.855266,273.088099,252.954138,985.140468,16521089
3,15561238,4.792836,1.272056,212.937925,156.223674,0.890723,0.705203,273.126062,259.557456,989.314698,15561238
4,14085613,4.819028,2.239776,253.690478,230.950833,1.201308,0.688723,273.393551,269.375118,984.483658,14085613
...,...,...,...,...,...,...,...,...,...,...,...
507,13840000,5.383687,1.529497,225.171796,156.567743,1.159049,0.929777,273.470298,263.146395,985.801841,13840000
508,12660000,4.777020,2.451088,260.956781,229.604138,1.077705,0.575494,273.888627,271.557464,985.606181,12660000
509,10710000,4.771453,4.176458,294.931709,244.702852,1.585094,0.360146,275.245088,278.360921,978.436682,10710000
510,7690000,4.754014,5.193846,313.311345,196.774631,1.957911,0.248088,277.253314,280.413695,978.293303,7690000


In [ ]:
#Creating datasets with lag of 1 to 6 months
df1 = df.assign(label = df.label.shift(-1)).drop(df.index[-1])
df2 = df1.assign(label = df1.label.shift(-1)).drop(df1.index[-1])
df3 = df2.assign(label = df2.label.shift(-1)).drop(df2.index[-1])
df4 = df3.assign(label = df3.label.shift(-1)).drop(df3.index[-1])
df5 = df4.assign(label = df4.label.shift(-1)).drop(df4.index[-1])
df6 = df5.assign(label = df5.label.shift(-1)).drop(df5.index[-1])
df1

,label,wind_10m,specific_humidity,LW_down,SW_down,rainfall,snowfall,sst,t2m,surface_pressure,sea_ice_extent
0,16378929.0,5.531398,0.811961,186.687054,3.127880,1.009872,0.892319,273.355237,250.388101,984.633032,15604191
1,16521089.0,5.328020,0.688896,174.794571,18.541594,0.920831,0.781347,273.121885,247.071202,983.980418,16378929
2,15561238.0,5.432511,0.916124,190.741933,67.690429,0.983327,0.855266,273.088099,252.954138,985.140468,16521089
3,14085613.0,4.792836,1.272056,212.937925,156.223674,0.890723,0.705203,273.126062,259.557456,989.314698,15561238
4,12653185.0,4.819028,2.239776,253.690478,230.950833,1.201308,0.688723,273.393551,269.375118,984.483658,14085613
...,...,...,...,...,...,...,...,...,...,...,...
506,13840000.0,5.494218,1.006108,194.683072,68.315949,1.135685,0.919100,273.414735,254.391240,977.764826,14640000
507,12660000.0,5.383687,1.529497,225.171796,156.567743,1.159049,0.929777,273.470298,263.146395,985.801841,13840000
508,10710000.0,4.777020,2.451088,260.956781,229.604138,1.077705,0.575494,273.888627,271.557464,985.606181,12660000
509,7690000.0,4.771453,4.176458,294.931709,244.702852,1.585094,0.360146,275.245088,278.360921,978.436682,10710000


In [ ]:
df2.columns

Index(['label', 'wind_10m', 'specific_humidity', 'LW_down', 'SW_down',
       'rainfall', 'snowfall', 'sst', 't2m', 'surface_pressure',
       'sea_ice_extent'],
      dtype='object')

In [ ]:
#data = np.array(df1)
target = df6['label'] #assign last column to be target variable
data = df6.iloc[:,:] #dropping last column from features

# print(data.shape)
# print(target.shape)
LEN_DATA = len(data) #total number of pixels
NUM_TRAIN = LEN_DATA - (100) #reserve last 30 months for testing



X_train = data[0:NUM_TRAIN]
y_train = target[0:NUM_TRAIN]

X_test = data[NUM_TRAIN:]
y_test=target[NUM_TRAIN:]



In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(data,target,test_size=0.3,) # .target.reshape(-1, 1)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(406, 11)
(406,)
(100, 11)
(100,)


In [ ]:
#printing out a random y_train value to tally it after min max scaling
print(X_train.head)
print(y_train.head)

<bound method NDFrame.head of           label  wind_10m  ...  surface_pressure  sea_ice_extent
0    10521427.0  5.531398  ...        984.633032        15604191
1     8183421.0  5.328020  ...        983.980418        16378929
2     7219913.0  5.432511  ...        985.140468        16521089
3     9428492.0  4.792836  ...        989.314698        15561238
4    11185959.0  4.819028  ...        984.483658        14085613
..          ...       ...  ...               ...             ...
401  12261353.0  4.588683  ...        982.794845        10965768
402  13913058.0  4.442885  ...        980.008459         7955719
403  14813126.0  4.704143  ...        979.376774         4712820
404  15159427.0  5.162643  ...        978.463490         3629315
405  14440180.0  5.279672  ...        984.954327         7086881

[406 rows x 11 columns]>
<bound method NDFrame.head of 0      10521427.0
1       8183421.0
2       7219913.0
3       9428492.0
4      11185959.0
          ...    
401    12261353.0
402    1

In [ ]:
# Formula for Bayesian Linear Regression (follows R formula syntax
formula = 'label ~ ' + ' + '.join(['%s' % variable for variable in X_train.columns[1:]])
formula

'label ~ wind_10m + specific_humidity + LW_down + SW_down + rainfall + snowfall + sst + t2m + surface_pressure + sea_ice_extent'

## Implementing Bayesian Linear Regression
### Create Model in PyMC3 and Sample from Posterior

In [ ]:
# Context for the model
with pm.Model() as normal_model:
    
    # The prior for the model parameters will be a normal distribution
    family = pm.glm.families.Normal()
    
    # Creating the model requires a formula and data (and optionally a family)
    pm.GLM.from_formula(formula = formula, data = X_train, family = family)
    
    # Perform Markov Chain Monte Carlo sampling
    normal_trace = pm.sample(draws=550, tune = 100)

The glm module is deprecated and will be removed in version 4.0
We recommend to instead use Bambi https://bambinos.github.io/bambi/
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  # This is added back by InteractiveShellApp.init_path()
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [sd, sea_ice_extent, surface_pressure, t2m, sst, snowfall, rainfall, SW_down, LW_down, specific_humidity, wind_10m, Intercept]


Sampling 2 chains for 100 tune and 550 draw iterations (200 + 1_100 draws total) took 8 seconds.
The acceptance probability does not match the target. It is 0.9123914048007968, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9059160838049067, but should be close to 0.8. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


### Examine Bayesian Linear Regression Results

In [ ]:

# Evalute the MCMC trace and compare to ml models
def evaluate_trace(trace, X_train, X_test, y_train, y_test):
    
    # Dictionary of all sampled values for each parameter
    var_dict = {}
    for variable in trace.varnames:
        var_dict[variable] = trace[variable]
        
    # Results into a dataframe
    var_weights = pd.DataFrame(var_dict)
    
    # Means for all the weights
    var_means = var_weights.mean(axis=0)
    
    # Create an intercept column
    X_test['Intercept'] = 1
    
    # Align names of the test observations and means
    names = X_test.columns[1:]
    X_test = X_test.iloc[:, 1:]
    var_means = var_means[names]
    
    # Calculate estimate for each test observation using the average weights
    #results = pd.DataFrame(index = X_test.index, columns = ['estimate'])
    results = []
    #i = 0
    for row in X_test.iterrows():
        #print(np.dot(np.array(var_means), np.array(row[1])))
        results = np.append(results, np.dot(np.array(var_means), np.array(row[1])))

    # Metrics
    results = results * (-1)
    print(results) 
    actual = np.array(y_test)
    errors = results - actual
    #mae = np.mean(abs(errors))
    rmse = np.sqrt(np.mean(errors ** 2))
    nrmse = rmse/np.mean(actual)
    r2 = r2_score(actual,results)
    
    print('Model  RMSE: {:.4f}\nModel NRMSE: {:.4f}\nModel R-Squared: {:.4f}'.format(rmse,nrmse,r2))
    #print('Model  RMSE: {:.4f}\n'.format(rmse))

    return rmse, results

In [ ]:
rmse, model_results = evaluate_trace(normal_trace, X_train, X_test, y_train, y_test)

In [ ]:
print(model_results)

In [ ]:
"""
#Save Test data vs predictions
from tempfile import TemporaryFile

#test_data = np.concatenate((inv_y_valid.reshape(-1,1),inv_testPred.reshape(-1,1)), axis=1)
np_file = TemporaryFile()
np.save('BLR_lag2.npy', results)
_ = np_file.seek(0)
"""